# Data Generation with Flatland

This notebook begins to outline how training datasets will be generated in Flatland. This notebook will evolve into a documentation of how to use the tool through a more formal and simplified API.

### Here's the idea

Some modern protein structure prediction approaches are a little bit complicated to implement. That's why it seems useful to have a simulator that can generate data at least of the same structure that researchers seek to use in such systems. Over time, these simulators can be improved progressively to add some minimal level of realism that should be helpful for initial model debugging. This might include for example modifying simulation parameters to enable a model to train effectively then returning to the more complex form once the simpler problem has been solved. Thus we hope to create a much more smooth path to solving the larger problem than is often followed by those seeking to solve it directly.

Further, even when training larger systems on real data it will be important that system components remain integrated and both the system and its individual components continue to function correctly. Simple toy test cases are often used for this purpose in software test engineering. But in the case of ML software engineering, sometimes it helps if these are slightly realistic. Even further, we are interested in understanding the potential of various additional sources of data to enhance the performance of structure prediction systems.

Below we will evolve populations of polymers using a trivially simple fitness metric and in the course of that retain a "genetic history" of the evolved populations. Then, we will compute structures for these polymers using Jax MD. For each "solved" structure we will compute a pairwise "residue" distance matrix and a vector of "bond" angles. Lastly, we will simulate a compound-protein interaction experiment again using Jax MD.


### Setup

Ensure the most recent version of Flatland in installed.

In [ ]:

!pip install git+git://github.com/cayley-group/flatland.git --quiet


In [31]:

from jax import jit, random, vmap
import jax.numpy as jnp

from flatland import evolution as evo
from flatland import datagen

from google.cloud import storage

import os

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

def get_destination_blob_path(filename, test_train_validation,
                              dataset_name, dataset_version):
  """Return a blob path."""
  ttv = ["test", "train", "validation"]
  assert test_train_validation in ttv
  return os.path.join(dataset_name, dataset_version,
                      test_train_validation, filename)


## Configuration

Here we'll configure our evolutionary simulations. We'll configure these to be very simple given that this is a demo - polymers of length 10 with elements from an alphabet of size 3. And just 10 population members for only 2 generations.

In [32]:

alphabet_size = 3
population_size = 10
genome_length = 10
mutation_rate = 0.15
num_generations = 2

bucket_name = "test-1234-abcd"
dataset_name = "flatland-vsmall"
dataset_version = "0.0.1"


## Run simulations

Now we'll evolve our polymer populations using a trivial fitness measure - i.e. how closely the average of the integer encodings of polymer elements come to 1.0! Here we could specify the fitness_fn to be one that simulates the polymers, computes their energy, and simply considered the simulated polymer energy to be a measure of fitness (energies are more negative for more energetically-stable strutures). Or likewise we could simulate the interaction of polymers with a set of compounds and define fitness as the selectivity of systems to be low energy only when including one or more target compounds.


In [25]:
from functools import partial

In [26]:

def fitness_mean_value_target(v, target_value=1.0):
  return 1 - jnp.abs(jnp.mean(v) - target_value) / target_value

@jit
def batched_fitness_mean_value_target(population):
  return vmap(fitness_mean_value_target)(population)

"""
Or,

def fitness_polymer_min_energy(v):
  return compute_energy(simulate(v))

@jit
def batched_polymer_min_energy(population):
  return vmap(fitness_polymer_min_energy)(population)

"""

key = random.PRNGKey(1)

# Construct a partial for the evolution function since this
# will be the same for train, test, and eval
evolve = partial(evo.evolve_with_mutation,
  fitness_fn=batched_fitness_mean_value_target,
  num_generations=num_generations,
  pop_size=population_size,
  genome_length=genome_length,
  mutation_rate=mutation_rate,
  alphabet_size=alphabet_size,
  keep_full_population_history=True)


#### Evolve the training population

In the future we could scale this up arbitrarily by doing as many of these simulations in parallel as we like and aggregating the result. This is analagous to simulating the independent evolution of polymer families that are evolutionarily distant.

In [13]:

key, subkey = random.split(key)
_, _, train_population = evolve(key)


Current average fitness: 0.57


In [14]:

train_dataset_path = datagen.compile_dataset_for_population(
    subkey, train_population)


INFO:absl:Generating examples for polymer population of size 10
INFO:absl:Finished processing 1 examples.
INFO:absl:Finished processing 2 examples.
INFO:absl:Finished processing 3 examples.
INFO:absl:Finished processing 4 examples.
INFO:absl:Finished processing 5 examples.
INFO:absl:Finished processing 6 examples.
INFO:absl:Finished processing 7 examples.
INFO:absl:Finished processing 8 examples.
INFO:absl:Finished processing 9 examples.
INFO:absl:Finished processing 10 examples.
INFO:absl:Finished generating examples, writing to disk.


In [33]:

train_destination_path = get_destination_blob_path(
    filename="data.pkl",
    test_train_validation="train",
    dataset_name=dataset_name,
    dataset_version=dataset_version)

upload_blob(bucket_name=bucket_name,
            source_file_name=train_dataset_path,
            destination_blob_name=train_destination_path)


File /tmp/tmp_rd58zz7 uploaded to flatland-vsmall/0.0.1/train/data.pkl.


#### Evolve the test population

For now, evolve a single test polymer population. This will be analagous to the problem of inferring the structure of a polymer that is evolutionarily distant from any we have seen before.

In some sense, this kind of generalization may not be necessary - as solvers may do well to just memmorize solutions to the kinds of structures that are known to occur in nature. One way to interpret recent success using evolutionary information for folding is that it does exactly this - cues solvers regarding how to re-use previously-accumulated knowledge about how certain subsequences fold. 

The benefit of such a test would be regarding completely novel polymers that are not homologous to anything currently known to occur in nature. Or at least which arise from a anciently-diverged part of the evolutionary tree from the one our model was trained on.

It would be feasible to construct a test set sharing a closer evolutionary history with the training populations by selecting polymers to hold out from these to use in testing - sharing alignments across both.


In [15]:

key, subkey = random.split(key)
_, _, test_population = evolve(key)


Current average fitness: 0.20999996


In [16]:

test_dataset_path = datagen.compile_dataset_for_population(
    subkey, test_population)


INFO:absl:Generating examples for polymer population of size 10
INFO:absl:Finished processing 1 examples.
INFO:absl:Finished processing 2 examples.
INFO:absl:Finished processing 3 examples.
INFO:absl:Finished processing 4 examples.
INFO:absl:Finished processing 5 examples.
INFO:absl:Finished processing 6 examples.
INFO:absl:Finished processing 7 examples.
INFO:absl:Finished processing 8 examples.
INFO:absl:Finished processing 9 examples.
INFO:absl:Finished processing 10 examples.
INFO:absl:Finished generating examples, writing to disk.


In [36]:

test_destination_path = get_destination_blob_path(
    filename="data.pkl",
    test_train_validation="test",
    dataset_name=dataset_name,
    dataset_version=dataset_version)

upload_blob(bucket_name=bucket_name,
            source_file_name=test_dataset_path,
            destination_blob_name=test_destination_path)


File /tmp/tmp_9x8gjn7 uploaded to flatland-vsmall/0.0.1/test/data.pkl.


#### Evolve the validation population

The same for the validation set as for the test set.

In [5]:

key, subkey = random.split(key)
_, _, validation_population = evolve(key)


Current average fitness: 0.48999995


In [6]:

validation_dataset_path = datagen.compile_dataset_for_population(
    subkey, validation_population)


INFO:absl:Generating examples for polymer population of size 10
/opt/conda/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py:2796: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.int64'> requested in array is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax._check_user_dtype_supported(dtype, "array")
/opt/conda/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py:1875: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in sum is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax._check_user_dtype_supported(dtype, name)
INFO:absl:Finished proces

In [39]:

validation_destination_path = get_destination_blob_path(
    filename="data.pkl",
    test_train_validation="validation",
    dataset_name=dataset_name,
    dataset_version=dataset_version)

upload_blob(bucket_name=bucket_name,
            source_file_name=validation_dataset_path,
            destination_blob_name=validation_destination_path)


File /tmp/tmpukuh5n0o uploaded to flatland-vsmall/0.0.1/validation/data.pkl.
